# Illustrates how to program the trigger:

### Connect:

In [ ]:
import phywhisperer.usb as pw
phy = pw.Usb()
phy.con(program_fpga=True)
phy.set_power_source("host")

### Power-off target to allow PhyWhisperer to autodetect its speed:
Ensure the target device is connected to the PhyWhisperer.
We'll turn off the target's power so that the PhyWhisperer can be programmed before turning the target back on. We do this because we'll be capturing what the target does when it's first turned on.

In [ ]:
import time
phy.set_power_source("off")
time.sleep(0.5)
phy.reset_fpga()

### (Optionally) Manually set USB speed:
By default, PhyWhisperer will auto-detect the target's USB speed when the target is connected or powered-up, so this should not be necessary.

In [ ]:
phy.set_usb_mode('auto')

### Program trigger parameters:
Here we enable the output trigger pulse, we set the number of pulses that are issued, and we set the width and delay parameters of each pulse.

In this example we program PhyWhisperer to issue 2 trigger pulses after a pattern match:
- the first pulse is immediately after the pattern match and 2 cycles wide;
- the second pulse is 10 cycles after the first pulse and 4 cycles wide.

(See `help(phy.set_trigger)` for more examples.)

In [ ]:
phy.set_trigger(num_triggers=2, delays=[0,10], widths=[2,4])

### Arm the PhyWhisperer:
You should see the blue ARM LED turn on to reflect the armed status.

In [ ]:
phy.arm()

### Program the pattern match:

In [ ]:
phy.set_pattern(pattern=[0x2d, 0x00], mask=[0xff, 0xff])

### Tell PhyWhisperer how many events to capture:
Maximum is 8188.

In [ ]:
phy.set_capture_size(500)

### Power up the target:
Now that PhyWhisperer is programmed, power up the target. PW should auto-detect the correct speed, then the trigger should occur immediately.
The trigger can be observed on the IO4 pin of the ChipWhisperer connector and on the "Trig Out" MCX connector.

In [ ]:
phy.set_power_source("host")
#Let device enumerate
time.sleep(1.0)

### Ensure correct USB speed was detected:
If the assertion fails, try setting the USB speed manually with set_usb_mode().

In [ ]:
assert (phy.get_usb_mode() == 'FS')
#assert (phy.get_usb_mode() == 'LS')
#assert (phy.get_usb_mode() == 'HS')

### Read what was captured:

In [ ]:
raw = phy.read_capture_data()

### Interpret the captured data:
The pattern match byte which triggered the captured isn't recorded; let's add it back it so that the USB data can be properly interpreted.
Then, split the raw data into packets, and print them:

In [ ]:
phy.addpattern = True
packets = phy.split_packets(raw)
phy.print_packets(packets)

### Iterate and generate a trigger pulse of increasing width:

In [ ]:
import time
start_time = time.time()
for i in range (16):
    print("Iteration %d: " % i, end='')
    # toggle target power on and off:
    phy.set_power_source("off")
    time.sleep(0.1)
    # set trigger parameters:
    phy.set_trigger(delays=[0], widths=[2**i])
    # arm:
    phy.arm()
    phy.set_power_source("host")    
    phy.wait_disarmed()
    print("done")
    time.sleep(0.1)
print("Elapsed time: %d seconds" % (time.time()-start_time))   

### Capture and trigger delays are independent:
When the trigger delay is set with `set_trigger(delays=[X])`, a capture delay is automatically set to match the first trigger delay (i.e. the capture will start when the first trigger pulse is issued).

It's also possible to set the capture delay independently of the trigger delay with `set_capture_delay()`. Just remember that because `set_trigger()` sets a matched capture delay, `set_capture_delay()` must be called after, otherwise the capture delay will get overwritten.

Here's a simple example which compares the data received with a capture delays of 0 and 200 clock cycles. YMMV but with an FS target, I observe that the delayed capture data starts 3 data bytes later:

In [ ]:
datatimes = [[], []]
databytes = [[], []]
stattimes = [[], []]
statbytes = [[], []]
for capture_delay in (0,1):
    # toggle target power on and off:
    phy.set_power_source("off")
    time.sleep(0.1)
    # set trigger parameters:
    phy.set_trigger(delays=[phy.ms_trigger(4)], widths=[1])
    if capture_delay == 0:
        phy.set_capture_delay(0)
    else:
        phy.set_capture_delay(200)
    # arm:
    phy.arm()
    phy.set_power_source("host")    
    phy.wait_disarmed()
    time.sleep(0.1)
    raw = phy.read_capture_data()
    phy.addpattern = False
    datatimes[capture_delay], databytes[capture_delay], stattimes[capture_delay], statbytes[capture_delay] = phy.split_data(raw)

print("Immediata capture data    Delayed capture data")
for i in range(10):
    print("%4s                      %4s" % (hex(databytes[0][i]), hex(databytes[1][i])))

In [ ]:
phy.close()